# Grid search for hyperparameter tunning

# Import and setup


In [61]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, cohen_kappa_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from IPython.display import display, clear_output
import time
import itertools

from moe_model import MoE as MoE_raw, MLP as MoE_Expert

# --- Helper class to modify the MoE Expert to output raw logits ---
# This is necessary because the CrossEntropyLoss function expects logits, not probabilities.
class MoE_Expert_Logits(MoE_Expert):
    def __init__(self, input_size, output_size, hidden_size):
        super().__init__(input_size, output_size, hidden_size)
        # Replace the final softmax layer with an identity layer
        self.soft = nn.Identity()


# Trainning and evaluation functions

In [62]:
def train_and_evaluate_trial(params, X_train, y_train, X_val, y_val):
    """
    Trains and evaluates a single trial of the MoE model with a given set of hyperparameters.
    """
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    # --- Model Configuration ---
    model = MoE_raw(
        input_size=X_train.shape[1],
        output_size=2,
        num_experts=params['num_experts'],
        hidden_size=params['hidden_size'],
        k=params['k']
    )
    model.experts = nn.ModuleList([
        MoE_Expert_Logits(
            input_size=X_train.shape[1],
            output_size=2,
            hidden_size=params['hidden_size']
        ) for _ in range(model.num_experts)
    ])
    model.to(device)
    
    # --- Data Preparation ---
    X_train_tensor = torch.FloatTensor(X_train).to(device)
    y_train_tensor = torch.LongTensor(y_train).to(device)
    X_val_tensor = torch.FloatTensor(X_val).to(device)
    y_val_tensor = torch.LongTensor(y_val).to(device)

    # --- Training with Early Stopping ---
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    patience = 5
    best_val_loss = float('inf')
    patience_counter = 0
    max_epochs = 75

    for epoch in range(max_epochs):
        model.train()
        optimizer.zero_grad()
        y_pred, aux_loss = model(X_train_tensor)
        loss = criterion(y_pred, y_train_tensor) + aux_loss
        loss.backward()
        optimizer.step()
        
        model.eval()
        with torch.no_grad():
            y_val_pred, val_aux_loss = model(X_val_tensor)
            val_loss = criterion(y_val_pred, y_val_tensor) + val_aux_loss
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
        else:
            patience_counter += 1
        
        if patience_counter >= patience:
            break

    # --- Final Evaluation on Validation Set ---
    model.eval()
    with torch.no_grad():
        y_pred_tensor, _ = model(X_val_tensor)
        probas = nn.functional.softmax(y_pred_tensor, dim=1)
        _, predicted = torch.max(probas, 1)
        
        y_true = y_val_tensor.cpu().numpy()
        y_pred = predicted.cpu().numpy()
        y_score = probas[:, 1].cpu().numpy()
        
        # Get detailed metrics from classification report for class '1'
        report = classification_report(y_true, y_pred, output_dict=True, zero_division=0).get('1', {})

        return {
            'num_experts': params['num_experts'],
            'hidden_size': params['hidden_size'],
            'k': params['k'],
            'val_roc_auc': roc_auc_score(y_true, y_score),
            'val_accuracy': accuracy_score(y_true, y_pred),
            'val_f1_score': report.get('f1-score', 0),
            'val_precision': report.get('precision', 0),
            'val_recall': report.get('recall', 0),
            'val_cohen_kappa': cohen_kappa_score(y_true, y_pred)
        }


# Data Loading and Hyperparameter definition

In [63]:
# --- 1. Load and Prepare Data ---
try:
    print("🔹 Loading and preparing data...")
    df = pd.read_csv('input_moe_rfe_fulldataset.csv', low_memory=False)
    X = df.drop(columns=['Dementia Status'])
    y = df['Dementia Status']
    
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)
    
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)
    X_test_scaled = scaler.transform(X_test)
    
    print(f"Data loaded successfully. Train shape: {X_train.shape}")
except FileNotFoundError:
    print("\n⚠️ ERROR: 'input_data.csv' not found. Please run the feature extractor first.")

# --- 2. Define the Hyperparameter Grid ---
param_grid = {
    'num_experts': [3, 4, 6, 8],
    'hidden_size': [16, 32, 64],
    'k': [2, 3, 4]
}

# Create all possible combinations
all_params = list(itertools.product(
    param_grid['num_experts'],
    param_grid['hidden_size'],
    param_grid['k']
))

# Filter out invalid combinations where k > num_experts
valid_params = [
    {'num_experts': p[0], 'hidden_size': p[1], 'k': p[2]}
    for p in all_params if p[2] <= p[0]
]

print(f"\n🔹 Starting Grid Search. Total combinations to test: {len(valid_params)}")


🔹 Loading and preparing data...
Data loaded successfully. Train shape: (1297, 31)

🔹 Starting Grid Search. Total combinations to test: 33


# Run grid search and final evaluation

In [64]:
# --- 3. Run the Grid Search ---
results = []
for i, params in enumerate(valid_params):
    print(f"\n--- Testing Combination {i+1}/{len(valid_params)} ---")
    print(f"Parameters: {params}")
    
    result = train_and_evaluate_trial(params, X_train_scaled, y_train.values, X_val_scaled, y_val.values)
    results.append(result)
    
    # Live update of results
    clear_output(wait=True)
    results_df = pd.DataFrame(results).sort_values('val_roc_auc', ascending=False)
    print("✅ Intermediate Tuning Results (Validation Set):")
    display(results_df)

# --- 4. Final Evaluation on Test Set ---
print("\n\n" + "="*40 + "\n✅ FINAL EVALUATION ON HELD-OUT TEST SET\n" + "="*40)

# Get the best parameters from the grid search
best_params = results_df.iloc[0].to_dict()
print("🏆 Best Hyperparameters found (based on validation ROC-AUC):")
print(f"  - Number of Experts: {int(best_params['num_experts'])}")
print(f"  - Hidden Size: {int(best_params['hidden_size'])}")
print(f"  - Top K: {int(best_params['k'])}")

# Combine training and validation data
X_train_val = np.concatenate((X_train_scaled, X_val_scaled), axis=0)
y_train_val = np.concatenate((y_train.values, y_val.values), axis=0)

print("\nRetraining the best model on combined Train+Validation data...")

# Create the final model with the best parameters
final_model = MoE_raw(
    input_size=X_train_val.shape[1],
    output_size=2,
    num_experts=int(best_params['num_experts']),
    hidden_size=int(best_params['hidden_size']),
    k=int(best_params['k'])
)
final_model.experts = nn.ModuleList([
    MoE_Expert_Logits(
        input_size=X_train_val.shape[1],
        output_size=2,
        hidden_size=int(best_params['hidden_size'])
    ) for _ in range(final_model.num_experts)
])

# Train the final model and evaluate on the test set
final_results = train_and_evaluate_trial(
    {'num_experts': int(best_params['num_experts']), 'hidden_size': int(best_params['hidden_size']), 'k': int(best_params['k'])},
    X_train_val, y_train_val, X_test_scaled, y_test.values
)

# --- Print all requested metrics for the final evaluation ---
print("\n--- Final Performance on Test Set ---")
print(f"  - Accuracy:    {final_results['val_accuracy']:.4f}")
print(f"  - ROC-AUC:     {final_results['val_roc_auc']:.4f}")
print(f"  - F1-Score:    {final_results['val_f1_score']:.4f}")
print(f"  - Precision:   {final_results['val_precision']:.4f}")
print(f"  - Recall:      {final_results['val_recall']:.4f}")
print(f"  - Cohen Kappa: {final_results['val_cohen_kappa']:.4f}")


✅ Intermediate Tuning Results (Validation Set):


,num_experts,hidden_size,k,val_roc_auc,val_accuracy,val_f1_score,val_precision,val_recall,val_cohen_kappa
23,6,64,4,0.784956,0.708633,0.730897,0.687500,0.780142,0.415997
22,6,64,3,0.784335,0.723022,0.745875,0.697531,0.801418,0.444721
7,4,16,3,0.783921,0.694245,0.723127,0.668675,0.787234,0.386775
5,3,64,3,0.783455,0.723022,0.745875,0.697531,0.801418,0.444721
14,4,64,4,0.783248,0.712230,0.735099,0.689441,0.787234,0.423147
10,4,32,3,0.782782,0.719424,0.743421,0.693252,0.801418,0.437451
17,6,16,4,0.781695,0.719424,0.740000,0.698113,0.787234,0.437685
20,6,32,4,0.781125,0.719424,0.743421,0.693252,0.801418,0.437451
19,6,32,3,0.777553,0.701439,0.731392,0.672619,0.801418,0.401080
2,3,32,2,0.776725,0.712230,0.736842,0.687117,0.794326,0.423027




✅ FINAL EVALUATION ON HELD-OUT TEST SET
🏆 Best Hyperparameters found (based on validation ROC-AUC):
  - Number of Experts: 6
  - Hidden Size: 64
  - Top K: 4

Retraining the best model on combined Train+Validation data...

--- Final Performance on Test Set ---
  - Accuracy:    0.7482
  - ROC-AUC:     0.8178
  - F1-Score:    0.7619
  - Precision:   0.7273
  - Recall:      0.8000
  - Cohen Kappa: 0.4960
